In [ ]:
import numpy as np
import scipy as sp
import scipy.linalg
import scipy.stats
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
from sklearn.metrics import r2_score
import pickle
import time
from scipy.optimize import fsolve
from scipy.optimize import least_squares
from scipy.optimize import minimize
from scipy import sparse
import os.path
from scipy.interpolate import splrep, splev

Suppose M is the underlying matrix, X is the observed matrix. Noise to signal ratio is denoted by
$$
\sqrt{\frac{\|P_{\Omega}(X-M)\|_{F}^2}{\|P_{\Omega}(M)\|_{F}^2}} 
$$

In [ ]:
def noise_to_signal(X, M, Ω):
    return np.sqrt(np.sum((Ω*X - Ω*M)**2) / np.sum((Ω*M)**2))

In [ ]:
def abs_mean(X, M, Ω):
    return np.sum(np.abs((X-M)*Ω)) / np.sum(Ω)

In [ ]:
def entry_Frobenious(Mhat, M):
    return np.sqrt(np.sum((Mhat - M)**2))

In [ ]:
def entry_max(Mhat, M):
    return np.max(np.abs(Mhat - M))

In [ ]:
def count_AUC(FPR, TPR):
    return np.sum((TPR[1:] + TPR[:-1])*(FPR[1:]-FPR[:-1])/2)

#unit_test of count_AUC
#FPR = np.linspace(0, 1, 100)
#TPR = 1 - (1-FPR)**2 
#print(count_AUC(FPR, TPR))

Returns
$$
SVD(M)_r
$$

In [ ]:
## least-squares solved via single SVD
def SVD(M,r): #input matrix M, approximating with rank r
    u,s,vh = np.linalg.svd(M, full_matrices=False) #s is diag
    X = u[:,:r].dot(np.diag(np.sqrt(s[:r])))
    Y = vh[:r,:].T.dot(np.diag(np.sqrt(s[:r])))
    return X.dot(Y.T)

Returns $$\operatorname{argmin}_A \|A\|_1 + \gamma\|P_\Omega (A - X)\|_F^2$$

Indeed
\begin{align}
A_{ij} = \begin{cases} 0 & if |X_{ij}| < \frac{1}{2\gamma}~or~(i,j) \notin \Omega \\ (X_{ij} - sign(X_{ij})\frac{1}{2\gamma}) & otherwise \end{cases} 
\end{align}

In [ ]:
def l1_prox(X,Ω,γ):
    return Ω * np.sign(X) * np.maximum(0,np.abs(X)-.5/γ)

Returns $$\operatorname{argmin}_M \|M\|_* + \frac{1}{2\gamma}\|M - X\|_F^2$$

In [ ]:
def soft_threshold(X,γ):
    U,s,VT = np.linalg.svd(X, full_matrices=False)
    S_threshold = np.diag(np.maximum(0,s-γ))
    return U.dot(S_threshold).dot(VT)

Returns $$\operatorname{argmin}_M \|M\|_* + \frac{1}{2\gamma}\|P_\Omega(M - X)\|_F^2$$

In [ ]:
def soft_impute(X,Ω,γ,convergence_threshold=.001, debug=False):
    M_old = np.zeros(X.shape)
    M_new = soft_threshold(Ω*X + (1-Ω)*M_old,γ)
    while  np.sum((M_old-M_new)**2) >= convergence_threshold * np.sum(M_old**2):
        M_old = M_new
        M_new = soft_threshold(Ω*X + (1-Ω)*M_old,γ)
        #print(np.sum((M_old-M_new)**2), np.sum(M_old**2))
        #print(γ)
        if (np.sum(M_old**2)<1e-6):
            break
        if (debug):
            print(np.linalg.matrix_rank(M_new))
    return M_new

MLE estimator of p and alpha:
$$
\max_{p, \alpha} \sum_{ij} \log\left\{P\left(X|\frac{M}{1-p+p*\alpha}, p, \alpha\right)\right\}
$$

For an algorithm, if $t_{ij}$ is the probability to claim anomaly at the entry $(i, j)$, then
\begin{align}
TPR &= \frac{\sum_{ij} t_{ij} P(anomaly|X_{ij}) }{\sum_{ij} P(anomaly|X_{ij})} \\
FPR &= \frac{\sum_{ij} t_{ij} P(not~~anomaly|X_{ij})}{\sum_{ij} P(not~~anomaly|X_{ij})}
\end{align}
where 
\begin{align}
P(anomaly|X_{ij}) 
&= \frac{p\frac{(\alpha M^{*})^{X}}{X!}e^{-\alpha M^{*}}}{(1-p)\frac{(M^{*})^{X}}{X!}e^{-M^{*}}+p\frac{(\alpha M^{*})^{X}}{X!}e^{-\alpha M^{*}}}\\
&= \frac{p(\alpha)^{X} e^{-\alpha M^{*}}}{(1-p)e^{-M^{*}}+p(\alpha)^{X} e^{-\alpha M^{*}}} 
\end{align}

In [ ]:
class Linear_decrease_noise_model:
    '''
        not done yet
    '''

    def __init__(self, p, alpha, p_range = (1e-5, 0.4), one_over_exp_range = (1e-5, 0.5)):
        self.p = p
        self.alpha = alpha

    def add_noise(self, M0):

        anomaly_set = np.random.binomial(1, self.p, M0.shape)
        
        A = anomaly_set * self.alpha + (1-anomaly_set)

        X = np.random.poisson(M0*A)

        return anomaly_set, X

    def MLE_estimate(self, M, X, Ω, debug=False):
        def f(x):
            p = x[0]
            alpha = x[1]
            M_star = M / (1- p+alpha*p)
            A = X*np.log(alpha) + (1-alpha)*M_star
            logA = (A<20)*np.log((1-p)+p*np.exp(np.minimum(A, 20))) + (A>=20)*(A+np.log(p))
            #print(p, alpha)
            #print(- (np.sum(Ω*logA) - np.sum(Ω*X)*np.log(1-p+alpha*p)+np.sum(-Ω*M_star)))
            #print(p, alpha, np.sum(Ω*logA), - np.sum(Ω*X)*np.log(1-p+alpha*p), np.sum(-Ω*M_star))
            return - (np.sum(Ω*logA) - np.sum(Ω*X)*np.log(1-p+alpha*p) + np.sum(-Ω*M_star))
    
        
        res = minimize(f, (0.5, 0.5), bounds = ((1e-5, 0.5), (1e-5, 0.5)))
        print(res)
        return res.x[0], res.x[1]

    
    def posterior_anomaly_processing(self, M, X, Ω):
        M = np.maximum(M, 0) + 1e-9
        A = X*np.log(self.alpha) + (1-self.alpha)*M
        T1 = (A>30)*1e20 + (A<=30)*self.p*np.exp(np.minimum(A, 30))
        T3 = Ω * T1 / (T1 + 1-self.p)
        return np.reshape(T3, -1)

#     def unit_test_MLE_estimator():
#     ## Synthetic experiments
#     n = 300
#     n1 = 300
#     n2 = 300
#     r = 5
#     prob_missing = 1#10.0*r/n
#     p = 0.1
#     alpha = 0.1
#     exp_rate = 10
#     mean_value = 3

#     #np.random.seed(1)
#     U = np.random.gamma(1, 2, (n, r))
#     V = np.random.gamma(1, 2, (n, r))
#     M0 = U.dot(V.T)
#     M0 = M0 / np.mean(M0) * mean_value
#     #M0 = np.random.normal(size=(n, r)).dot(np.random.normal(size=(r, n)))
#     #u, s, vh = np.linalg.svd(M0, full_matrices=True)
#     #print(sorted(s))

#     Ω = np.random.binomial(1,prob_missing,(n,n))
#     #M0 = M0 * Ω 
#     #A = np.random.binomial(1,p,(n,n)) * (1-alpha) * M0
#     anomaly_set = np.random.binomial(1,p,(n,n))
#     A = anomaly_set * np.random.exponential(scale=1/exp_rate, size = (n, n)) + (1-anomaly_set)

#     X = np.random.poisson(M0*A)

#     print('true p is {} and true exp_rate is {}'.format(p, exp_rate))

#     p_est, exp_est = MLE_estimate_exponential_model(M0*(1-p+p/exp_rate), X, Ω, debug=False)
#     print('with M0, the p_est is {} and the exp_est is {}'.format(p_est, exp_est))

#     M = SVD(X*Ω, r)
#     M = np.maximum(M, 1e-8)
#     p_est, exp_est = MLE_estimate_exponential_model(M, X, Ω, debug=False)
#     print('with SVD, the p_est is {} and the exp_est is {}'.format(p_est, exp_est))

#     gamma = 70
#     M = soft_impute(X, Ω, gamma,convergence_threshold=1e-5)
#     M = np.maximum(M, 1e-8)
#     print('with soft_impute, the rank of M is {}, abs mean between M and M0 is {}'.format(np.linalg.matrix_rank(M), abs_mean(M, M0, Ω)))
#     p_est, exp_est = MLE_estimate_exponential_model(M, X, Ω, debug=False)
#     print('with soft_impute, the p_est is {} and the exp_est is {}'.format(p_est, exp_est))

# unit_test_MLE_estimator()

In [ ]:
class Exponential_noise_model:

    def __init__(self, p, exp_rate, p_range = (1e-5, 0.4), one_over_exp_range = (1e-5, 0.5)):
        self.p = p
        self.exp_rate = exp_rate
        self.p_range = p_range
        self.one_over_exp_range = one_over_exp_range

    def add_noise(self, M0):

        anomaly_set = np.random.binomial(1, self.p, M0.shape)
        
        A = anomaly_set * np.random.exponential(scale=1/self.exp_rate, size = M0.shape) + (1-anomaly_set)

        X = np.random.poisson(M0*A)

        return anomaly_set, X

    def counting_match_estimate(self, M, X, Ω, debug=False):
        def f(x):
            p = x[0]
            exp_rate = 1/x[1]
            M_star = M / (1- p+p/exp_rate)

            n0 = np.sum(Ω*(exp_rate / (M_star + exp_rate) * p + (1-p) * np.exp(-M_star))) / np.sum(Ω)
            n1 = np.sum(Ω*(exp_rate*M_star / (M_star + exp_rate)**2 * p + (1-p) * np.exp(-M_star)*M_star)) / np.sum(Ω) + n0
            n2 = np.sum(Ω*(exp_rate*(M_star**2) / ((M_star + exp_rate)**3) * p + (1-p) * np.exp(-M_star)*(M_star**2)/2)) / np.sum(Ω) + n1

            real_n0 = np.sum(Ω*(X==0)) / np.sum(Ω)
            real_n1 = np.sum(Ω*(X==1))/ np.sum(Ω) + real_n0
            real_n2 = np.sum(Ω*(X==2))/ np.sum(Ω) + real_n1
            if (debug):
                print(n0, real_n0, n1, real_n1, n2, real_n2)
            return np.sqrt((n0-real_n0)**2 + (n1-real_n1)**2 + (n2 - real_n2)**2)
        res = minimize(f, (self.p, 1/self.exp_rate), bounds = (self.p_range, self.one_over_exp_range))
        print(res)
        print('estimate hyper-parameter is p = {}, real_p is {}, exp_rate = {}, real_exp_rate is {}'.format(res.x[0], self.p, 1/res.x[1], self.exp_rate))
        return res.x[0], 1/res.x[1]

        
    def MLE_estimate(self, M, X, Ω, debug=False):
        def f(x):
            p = x[0]
            exp_rate = 1/x[1]
            M_star = M / (1- p+p/exp_rate)
            A = (X+1)*np.log(M_star+exp_rate) - M_star - scipy.special.gammaln(X+1)
            logA = (A<=30)*np.log(p*exp_rate + (1-p)*np.exp(np.minimum(A, 30))) + (A>30)*(A+np.log(1-p))
            #print(p, alpha)
            #print(- (np.sum(Ω*logA) - np.sum(Ω*X)*np.log(1-p+alpha*p)+np.sum(-Ω*M_star)))
            if (debug):
                print(p, exp_rate, np.sum(Ω*logA), np.sum(Ω*X*np.log(M_star/(M_star+exp_rate))) , -np.sum(Ω*np.log(M_star+exp_rate)))
            return - (np.sum(Ω*logA) + np.sum(Ω*X*np.log(M_star/(M_star+exp_rate))) - np.sum(Ω*np.log(M_star+exp_rate)))

        # m_min = 1e9
        # for p1 in np.linspace(1e-5, 0.5, num=30):
        #     for p2 in np.linspace(1e-5, 0.9, 30):
        #         tmp = f([p1, p2])
        #         if (tmp < m_min):
        #             if (debug):
        #                 print(p1, p2, tmp)
        #             m_min = tmp
        #             ans = [p1, p2]
        # return ans[0], 1/ans[1]
        # print(ans)
        #M = M0*(1-0.3+0.3*0.5)
        
        res = minimize(f, (0.1, 0.1), bounds = (self.p_range, self.one_over_exp_range))
        if (debug):
            print(res)
            print('estimate hyper-parameter is p = {}, real_p is {}, exp_rate = {}, real_exp_rate is {}'.format(res.x[0], self.p, 1/res.x[1], self.exp_rate))
        return res.x[0], 1/res.x[1]

    def posterior_anomaly_processing(self, M, X, Ω):
        M = np.maximum(M, 0) + 1e-9
        A = (X+1)*np.log(M+self.exp_rate) - M - scipy.special.gammaln(X+1)
        #some precision trick, we require p*exp_rate / ((1-p)*1e20 + p*exp_rate) \approx 0
        T1 = (A<=50)*(1-self.p)*np.exp(np.minimum(A, 50)) + (A>50)*1e30
        T3 = Ω * self.p*self.exp_rate / (T1 + self.p*self.exp_rate)
        return np.reshape(T3, T3.shape[0]*T3.shape[1])

    
    def evaluate_FPR_TPR(self, A_est, M, X, Ω):
        M = np.maximum(M, 0) + 1e-9
        A = (X+1)*np.log(M+self.exp_rate) - M - scipy.special.gammaln(X+1)
        #some precision trick, we require p*exp_rate / ((1-p)*1e20 + p*exp_rate) \approx 0
        T1 = (A<=50)*(1-self.p)*np.exp(np.minimum(A, 50)) + (A>50)*1e30
        posterior_anomaly = Ω * self.p*self.exp_rate / (T1 + self.p*self.exp_rate)

        TPR = np.sum((A_est != 0) * Ω * posterior_anomaly) / np.sum(Ω * posterior_anomaly)
        FPR = np.sum((A_est != 0) * Ω * (1 - posterior_anomaly)) / np.sum(Ω * (1-posterior_anomaly))

        return FPR, TPR


In [ ]:
class Experiment:
    
    def __init__(self, n1=1, n2=1, r=1, mean_value=1, prob_observing=1):

        self.synthetic_data(n1, n2, r, mean_value, prob_observing)

    def synthetic_data(self, n1, n2, r, mean_value, prob_observing):

        self.n1 = n1
        self.n2 = n2
        self.r = r
        self.mean_value = mean_value
        self.prob_observing = prob_observing
        U = np.random.gamma(1, 2, (n1, r))
        V = np.random.gamma(1, 2, (n2, r))
        M0 = U.dot(V.T)

        self.M0 = M0 / np.mean(M0) * mean_value

        self.Ω = np.random.binomial(1, prob_observing, (n1,n2))

        self.data_type = 'synthetic'

    def real_data(self):
        ## Real data experiments

        M_i = np.genfromtxt('data/wslr_unit_sales_matrix.csv',delimiter=',',filling_values=0)
        M_i = M_i[1:, 1:]
        Ω = 1-(M_i==0)

        rows = np.sum(Ω,axis=1) >= 30
        cols = np.sum(Ω,axis=0) >= 30
        #print(np.min(np.sum(Ω[rows].T[cols].T,axis=1)))
        #print(np.min(np.sum(Ω[rows].T[cols].T,axis=0)))

        M_i = M_i[rows].T[cols].T
        Ω = Ω[rows].T[cols].T
        print(np.sum(M_i)/np.sum(Ω))
        print(Ω.shape)
        print(np.sum(Ω))
        print(np.sum(Ω+1-Ω))

        self.M0 = M_i/4
        self.n1 = self.M0.shape[0]
        self.n2 = self.M0.shape[1]
        self.r = 30
        self.mean_value = np.sum(self.M0*Ω) / np.sum(Ω)
        self.prob_observing = np.sum(Ω) / (self.n1*self.n2)

        self.Ω = Ω
        self.data_type = 'beer_data'

    def add_noise(self, noise_model):
        self.anomaly_set, self.X = noise_model.add_noise(self.M0)
        self.noise = noise_model

Implementation of AD algorithm:
- Step 1: find $M$ using soft-impute 
$$
M = \operatorname{argmin}_M \|M\|_* + \frac{1}{2\gamma}\|P_\Omega(M - X)\|_F^2
$$

- Step 2: Using $M$ to esitimate $p$ and $\alpha$ and obtain $\hat{p}, \hat{\alpha}$ (MLE estimator)

- Step 3: Using $M, \hat{p}, \hat{\alpha}$ to estimate $P(anomaly|X_{ij})$

- Step 4: sorting $P(anomaly|X_{ij})$ from largest to the smallest, and claim the anomaly in this order 

In [ ]:
class Experiment(Experiment):

    def evaluate_ROC(self, posterior_anomaly, posterior_anomaly_est, Ω):
        '''
            sorting posterior_anomaly_est (1D array) from the largest to the smallest
            using the index to choose posterior_anomaly
            only entries in Ω is considered 

            Assuming the entries in posterior_anomaly_est is bounded by 1e20
        ''' 
        TΩ = np.reshape(Ω, -1)
        index = np.argsort(TΩ*(-posterior_anomaly_est) + (1-TΩ)*1e20)
        index = index[0:np.sum(Ω)]
        sorted_anomaly = posterior_anomaly[index]

        TPR = np.cumsum(sorted_anomaly) / np.sum(sorted_anomaly)
        FPR = np.cumsum(1-sorted_anomaly) / np.sum(1-sorted_anomaly)
        return FPR, TPR

    def ideal_ROC(self):
        posterior_anomaly = self.noise.posterior_anomaly_processing(self.M0, self.X, self.Ω)
        self.FPR_opt, self.TPR_opt = self.evaluate_ROC(posterior_anomaly, posterior_anomaly, self.Ω)
        self.posterior_anomaly = posterior_anomaly

    def AD_algorithm(self, gamma=1.0, r_constraint = 1e9, debug=False, do_SVD=False, using_ideal_parameter=False):
        
        if (do_SVD):
            M = SVD(self.Ω*self.X, r_constraint)
        else:
            while (True):
                M = soft_impute(self.X, self.Ω, gamma, convergence_threshold=1e-4, debug=debug)
                if (np.linalg.matrix_rank(M) > r_constraint):
                    gamma = gamma *1.1
                    if (debug):
                        print('gamma update with the rank ', np.linalg.matrix_rank(M))
                else:
                    break
            
        M = np.maximum(M, 1e-8)
        
        #p_est, exp_est = self.noise.counting_match_estimate(M, self.X, self.Ω, debug)
        p_est, exp_est = self.noise.MLE_estimate(M, self.X, self.Ω, debug)
        if (debug):
            print('error between M0 and X', abs_mean(self.X, self.M0, self.Ω))
            print('rank of M is', np.linalg.matrix_rank(M))
            print('error between M and M0 without correction', abs_mean(M, self.M0, self.Ω))
            print('error between M and M0 without correction outside omega', abs_mean(M, self.M0, 1-self.Ω))
            print('error between M and M0 with estimated correction', abs_mean(M/(1-p_est+p_est/exp_est), self.M0, self.Ω))
            #print('error between M and M0 with correct p and alpha', abs_mean(M/(1-p+p/exp_rate), self.M0, self.Ω))

        if (not using_ideal_parameter):
            self.posterior_anomaly_est = self.noise.posterior_anomaly_processing(M / (1 - p_est + p_est/exp_est), self.X, self.Ω)
        else:
            self.posterior_anomaly_est = self.noise.posterior_anomaly_processing(M / (1 - self.noise.p + self.noise.p/self.noise.exp_rate), self.X, self.Ω)

        self.FPR_AD, self.TPR_AD = self.evaluate_ROC(self.posterior_anomaly, self.posterior_anomaly_est, self.Ω)
        self.p_AD, self.exp_AD = p_est, exp_est
        self.M = M / (1 - p_est + p_est/exp_est)

    def PCA_with_M(self, M):

        self.posterior_anomaly_PCA = np.reshape(self.Ω*(np.abs(self.X-M)+1e-9), -1)
        #posterior_anomaly_PCA = np.reshape(Ω*(M0-X+100), -1)
        self.FPR_PCA_M0, self.TPR_PCA_M0 = self.evaluate_ROC(self.posterior_anomaly, self.posterior_anomaly_PCA, self.Ω)
        return self.FPR_PCA_M0, self.TPR_PCA_M0

Implementation of stable_pcp algorithm

 $$\operatorname{argmin}_{M,A} \|M\|_* + \lambda\|A\|_1 + \frac{\mu}{2}\|P_\Omega(M + A - X)\|_F^2$$

In [ ]:
class Experiment(Experiment):

    def stable_pcp(self, λ,μ,step_size=1,convergence_threshold=.001, debug=False, up_M = -1, up_A = -1):
        num_iter = 0
        
        X = self.X
        Ω = self.Ω

        M_old = soft_impute(X,Ω,1/μ)
        A_old = l1_prox(X-M_old,Ω,μ/λ/2)
        f = 1e20
        #for T in range(100):
        while (True):
            num_iter += 1
            G = μ*(M_old+A_old-X)
            M_new = soft_impute(M_old-step_size*G,Ω,step_size)
            A_new = l1_prox(A_old-step_size*G,Ω,1/2/λ/step_size)
            if (up_M > 0):
                M_new = np.maximum(np.minimum(M_new, up_M), -up_M)
                A_new = np.maximum(np.minimum(A_new, up_A), -up_A)
            if np.max(np.abs(M_old-M_new)) < convergence_threshold and np.max(np.abs(A_old-A_new)) < convergence_threshold:
                break
            else:
                M_old, A_old = M_new, A_new
                if (num_iter % 1000 == 0):
                    break
                if (num_iter % 100 == 0):
                    f_new = np.linalg.norm(M_new, 'nuc') + λ*np.sum(np.abs(A_new)) + 0.5*μ*np.sum((Ω*(self.X-A_new-M_new))**2)
                    if (np.abs(f_new - f) < 1e-3):
                        break
                    f = f_new
                    if (debug):
                        print(f)
                        print('abs_mean is', abs_mean(M_new, self.M0, Ω), 'deviation is', np.mean(M_new-self.M0))
                        print('rank is', np.linalg.matrix_rank(M_new))
                        print('step size is', step_size)
                
                if (num_iter % 10 == 0):
                    if (np.linalg.norm(M_new, 'nuc') + λ*np.sum(np.abs(A_new)) + 0.5*μ*np.sum((Ω*(self.X-A_new-M_new))**2) > 1e20):
                        step_size = step_size * 0.5
                        M_old = soft_impute(X,Ω,1/μ)
                        A_old = l1_prox(X-M_old,Ω,μ/λ/2)
        if (debug):
            print(num_iter)
            print(np.linalg.norm(M_new, 'nuc') + λ*np.sum(np.abs(A_new)) + 0.5*μ*np.sum((self.Ω*(self.X-A_new-M_new))**2))
            print('rank is', np.linalg.matrix_rank(M_new))
        return M_new,A_new

    def stable_pcp_withM0(M0, μ, λ):
        M_old = M0
        A_old = l1_prox(self.X-M_old,self.Ω,μ/λ/2)
        return M_old, A_old

    def Robust_PCA(self, gamma=50, r_constraint = -1, debug=False, num_points = 5, l_range = (-1, -1), up_M = -1, up_A = -1):
        ROC_PCA = [(0, 0), (1, 1)]

        if (l_range[0] == -1):
            l_range = (0.5, self.mean_value)
        numerate = np.linspace(l_range[0], l_range[1], num_points)
        self.RPCA_M = []
        for ratio in np.concatenate([numerate]):
            gamma = gamma * np.log(1+ratio)
            flag = 0
            while (True):
                M_est,A_est = self.stable_pcp(1/gamma*ratio,1/gamma,step_size=50,convergence_threshold=1e-4, debug=False, up_M=up_M, up_A=up_A)
                if (debug):
                    print('M0-M_est is', abs_mean(M_est, self.M0, self.Ω), 'and gamma is ', gamma, ' and l/mu is', ratio)
                    print('rank of M_est is ', np.linalg.matrix_rank(M_est))
                if (r_constraint < 0):
                    break
                if (np.linalg.matrix_rank(M_est) < r_constraint / 2):
                    if (flag == -1):
                        break
                    gamma = gamma / 1.1     
                else: 
                    if (np.linalg.matrix_rank(M_est) > r_constraint):
                        gamma = gamma + 0.2
                        flag = -1
                    else:
                        break
            gamma = gamma / np.log(1+ratio)
            #M_est,A_est = stable_pcp_withM0(X, Ω, ratio/100, 1/((mean_value*n/r/3)*1.7),step_size=50,convergence_threshold=1e-5, debug=True)
            #print('M0-M_est is', abs_mean(M_est, self.M0, self.Ω), ' and l/mu is', ratio)
            #print('rank of M_est is ', np.linalg.matrix_rank(M_est))
            self.RPCA_M.append(M_est)
            FPR, TPR = self.noise.evaluate_FPR_TPR(A_est, self.M0, self.X, self.Ω)
            print(FPR, TPR)
            ROC_PCA.append((FPR, TPR))

        ROC_PCA = sorted(ROC_PCA)
        self.FPR_PCA = np.array([t[0] for t in ROC_PCA])
        self.TPR_PCA = np.array([t[1] for t in ROC_PCA])

In [ ]:
class Experiment(Experiment):
    def DRMF(self, r, rate_anomalies):
        X = self.X
        Ω = self.Ω
        S = np.zeros_like(X)
        L = np.zeros_like(X)
        mm = np.sum(Ω)
        for t in range(40):
            L = SVD(Ω*(X - S - L) + L, r)
            #print(np.sort((np.abs(X - L)).reshape(-1)))
            thres = np.sort((np.abs(X - L)*Ω + (1-Ω)*1e8).reshape(-1))[int(mm*(1-rate_anomalies))]
            S = (X-L)*(np.abs(X-L) >= thres)
        self.DRMF_M = L
        self.FPR_DRMF, self.TPR_DRMF = self.PCA_with_M(L)
        #posterior_anomaly = self.noise.posterior_anomaly_processing(L, self.X, self.Ω)
        #self.FPR_DRMF, self.TPR_DRMF = self.evaluate_ROC(posterior_anomaly, self.posterior_anomaly_est, self.Ω)

In [ ]:
class Experiment(Experiment):

    def plot_ROC(self):
        print(count_AUC(self.FPR_opt, self.TPR_opt))
        print(count_AUC(self.FPR_AD, self.TPR_AD))
        plt.plot(self.FPR_opt, self.TPR_opt)
        plt.plot(self.FPR_AD, self.TPR_AD) 
        legend_list = [r'$\pi^{*}$', r'$\pi^{\mathrm{EW}}$']
        if (hasattr(self, 'FPR_PCA_M0')):
            print(count_AUC(self.FPR_PCA_M0, self.TPR_PCA_M0))
            plt.plot(self.FPR_PCA_M0, self.TPR_PCA_M0)
            legend_list.append('Stable PCP')
        if hasattr(self, 'FPR_PCA'):
            condition = (self.FPR_PCA <= self.TPR_PCA + 0.01)
            print(condition)
            print(count_AUC(self.FPR_PCA[condition], self.TPR_PCA[condition]))
            plt.plot(self.FPR_PCA[condition], self.TPR_PCA[condition])
            legend_list.append('Robust PCA')

        plt.legend(legend_list)
        plt.xlabel('FPR')
        plt.ylabel('TPR')
        plt.savefig(self.data_type + '.eps')

        f = open('data.npy', 'wb')
        np.save(f, self.FPR_opt)
        np.save(f, self.TPR_opt)
        np.save(f, self.FPR_AD)
        np.save(f, self.TPR_AD)
        np.save(f, self.FPR_PCA_M0)
        np.save(f, self.TPR_PCA_M0)
        #plt.title('n = {}, r = {}, mean_value(M*)= {}, p = {}, alpha = {}\n MLE_p = {:.3f}, MLE_alpha = {:.3f}'.format(n, r, mean_value, p, alpha, p_est, alpha_est))
        plt.show()

    def generate_data_for_heat_map(self, l1 = 3, l2 = 3, force=False):

        if (not force):
            return
        #if (not force and os.path.isfile('heat_map_data.npy')):
        #    return
        mean_value_set = np.linspace(0.1, 10, l1)
        alpha_set = np.linspace(0.1, 1, l2)

        AUC_opt = np.zeros((len(mean_value_set), len(alpha_set)))
        AUC_AD = np.zeros((len(mean_value_set), len(alpha_set)))
        AUC_PCA = np.zeros((len(mean_value_set), len(alpha_set)))
        for (i, mean_value) in enumerate(mean_value_set):
            for (j, alpha) in enumerate(alpha_set):
                self.synthetic_data(self.n1, self.n2, self.r, mean_value, self.prob_observing)
                noise = Exponential_noise_model(p = 0.2, exp_rate = 1/alpha, p_range = (0.01, 0.5), one_over_exp_range=(0.01, 0.8))
                self.add_noise(noise)
                self.ideal_ROC()
                self.AD_algorithm(gamma = (self.n1/self.r)*0.5, r_constraint = self.r, debug=False, do_SVD = False)

                

                #self.PCA_with_M(self.M0)
                self.Robust_PCA(gamma = (self.n2/self.r*0.5), r_constraint=self.r)
                #self.PCA_with_M(self.M0+0.3*np.sqrt(self.M0)*np.random.normal(size=self.M0.shape))


                AUC_opt[i, j] = count_AUC(self.FPR_opt, self.TPR_opt)
                AUC_AD[i, j] = count_AUC(self.FPR_AD, self.TPR_AD)
                #AUC_PCA[i, j] = count_AUC(self.FPR_PCA_M0, self.TPR_PCA_M0)
                AUC_PCA[i, j] = count_AUC(self.FPR_PCA, self.TPR_PCA)


        f = open('heat_map_data.npy', 'wb')
        np.save(f, mean_value_set)
        np.save(f, alpha_set)
        np.save(f, AUC_opt)
        np.save(f, AUC_AD)
        np.save(f, AUC_PCA)

    def plot_heat_map(self, file_path = "", file_name = 'heat_map_data'):

        def plot_fig(data, title):
            sns.heatmap(data, annot=True, xticklabels=alpha_set, yticklabels=mean_value_set)
            plt.xlabel(r'$\alpha$')
            plt.ylabel(r'$\bar{M}^{*}$')
            plt.title(title)
            plt.savefig(file_path+title+'.eps')
            plt.show()  

        f = open(file_path+file_name, 'rb')
        mean_value_set = np.round(np.load(f), decimals = 2)
        alpha_set = np.round(np.load(f), decimals = 2)
        AUC_opt = np.load(f)
        AUC_AD = np.maximum(np.load(f), 0.5)
        AUC_PCA = np.maximum(np.load(f), 0.5)
        print(AUC_PCA[0,1])

        plot_fig(AUC_opt, 'AUC of ideal')
        plot_fig(AUC_AD/AUC_opt, 'AUC of EW over AUC of ideal')
        plot_fig(AUC_PCA/AUC_opt, 'AUC of PCA over AUC of ideal')
        plot_fig(AUC_AD/AUC_PCA, 'AUC of EW over AUC of PCA')

    def plot_heat_map(self, file_path = '', file_name = ''):

        def plot_fig(data, title):
            sns.heatmap(data, annot=True, xticklabels=alpha_set, yticklabels=mean_value_set)
            plt.xlabel(r'$\alpha$')
            plt.ylabel(r'$\bar{M}^{*}$')
            plt.title(title)
            plt.savefig(file_path+title+'.eps')
            plt.show()  

        f = open(file_path+file_name, 'rb')
        mean_value_set = np.round(np.load(f), decimals = 2)
        alpha_set = np.round(np.load(f), decimals = 2)
        AUC_opt = np.load(f)
        AUC_AD = np.maximum(np.load(f), 0.5)
        AUC_PCA = np.maximum(np.load(f), 0.5)
        print(AUC_PCA[0,1])

        plot_fig(AUC_opt, 'AUC of ideal')
        plot_fig(AUC_AD/AUC_opt, 'AUC of EW over AUC of ideal')
        plot_fig(AUC_PCA/AUC_opt, 'AUC of PCA over AUC of ideal')
        plot_fig(AUC_AD/AUC_PCA, 'AUC of EW over AUC of PCA')

    def generate_data_for_scatter_plot(self, file_path = "", file_name = "", num_exper=200, r_range = [1, 10], mean_value_range = [1, 10], p_o_range = [0.5, 1], p_a_range = [0, 0.3], alpha_range = [0, 1], force=True):
        self.file_path = file_path
        self.file_name = file_name
        if (not force):
            return
        
        num_exper = 1000
        AUC_opt = np.zeros(num_exper)
        AUC_AD = np.zeros(num_exper)
        AUC_PCA = np.zeros(num_exper)
        AUC_RMF = np.zeros(num_exper)
        AUC_DRMF = np.zeros(num_exper)

        F_opt = np.zeros(num_exper)
        F_AD = np.zeros(num_exper)
        F_PCA = np.zeros(num_exper)
        F_RMF = np.zeros(num_exper)
        F_DRMF = np.zeros(num_exper)

        max_opt = np.zeros(num_exper)
        max_AD = np.zeros(num_exper)
        max_PCA = np.zeros(num_exper)
        max_RMF = np.zeros(num_exper)
        max_DRMF = np.zeros(num_exper)

        parameters = []

        start = 0
        for T in range(start, num_exper):
            r = np.random.randint(r_range[0], high=r_range[1]+1)
            mean = np.random.uniform(mean_value_range[0], mean_value_range[1])
            p_o = np.random.uniform(p_o_range[0], p_o_range[1])
            p_a = np.random.uniform(p_a_range[0], p_a_range[1])
            alpha = np.random.uniform(alpha_range[0], alpha_range[1])

            self.synthetic_data(self.n1, self.n2, r, mean, p_o)
            noise = Exponential_noise_model(p = p_a, exp_rate = 1/alpha, p_range = (0.01, 0.5), one_over_exp_range=(0.01, 0.9))
            self.add_noise(noise)
            self.ideal_ROC()
            self.AD_algorithm(gamma = self.n1/r*0.5, r_constraint = self.r, debug=False, do_SVD = False)
            self.DRMF(10, 0.3)
            AUC_opt[T] = count_AUC(self.FPR_opt, self.TPR_opt)
            AUC_AD[T] = np.maximum(0.5, count_AUC(self.FPR_AD, self.TPR_AD))
            AUC_DRMF[T] = np.maximum(0.5, count_AUC(self.FPR_DRMF, self.TPR_DRMF))
            F_opt[T] = 0
            F_AD[T] = entry_Frobenious(self.M, self.M0)
            F_DRMF[T] = entry_Frobenious(self.DRMF_M, self.M0)
            max_opt[T] = 0
            max_AD[T] = entry_max(self.M, self.M0)
            max_DRMF[T] = entry_max(self.DRMF_M, self.M0)

            # print('Frobenious AD', entry_Frobenious(self.M, self.M0))
            # print('Frobenious DRMF', entry_Frobenious(self.DRMF_M, self.M0))

            # print('max norm AD', entry_max(self.M, self.M0))
            # print('max norm DRMF', entry_max(self.DRMF_M, self.M0))
            #self.PCA_with_M(self.M0)

            self.Robust_PCA(gamma = (self.n2/r*0.5/2), r_constraint=self.r)
            AUC_PCA[T] = np.maximum(0.5, count_AUC(self.FPR_PCA, self.TPR_PCA))
            max_PCA[T] = 0
            F_PCA[T] = 0
            for M in self.RPCA_M:
                max_PCA[T] += entry_max(M, self.M0) / len(self.RPCA_M)
                F_PCA[T] += entry_Frobenious(M, self.M0) / len(self.RPCA_M)

            self.Robust_PCA(gamma = (self.n2/r*0.5/2), r_constraint=self.r, up_M = np.max(self.M0)*12, up_A = np.max(self.M0)*12)
            AUC_RMF[T] = np.maximum(0.5, count_AUC(self.FPR_PCA, self.TPR_PCA))
            max_RMF[T] = 0
            F_RMF[T] = 0
            for M in self.RPCA_M:
                max_RMF[T] += entry_max(M, self.M0) / len(self.RPCA_M)
                F_RMF[T] += entry_Frobenious(M, self.M0) / len(self.RPCA_M)
            #self.PCA_with_M(self.M0+0.3*np.sqrt(self.M0)*np.random.normal(size=self.M0.shape))


            #for M in self.RPCA_M:
            #    print('Frobenious PCA', entry_Frobenious(M, self.M0))

            #AUC_PCA[T] = count_AUC(self.FPR_PCA_M0, self.TPR_PCA_M0)
            #AUC_PCA[T] = count_AUC(self.FPR_PCA, self.TPR_PCA)

            parameters.append((r, mean, p_o, p_a, alpha))

            print('In the step {}, AUC_opt is {}, AUC_AD is {}, AUC_PCA is {}, AUC_DRMF is {}'.format(T, AUC_opt[T], AUC_AD[T], AUC_PCA[T], AUC_DRMF[T]))
            print('r is {}, mean is {}, p_o is {}, p_a is {}, alpha is {}'.format(r, mean, p_o, p_a, alpha))

            print('Frobenious norm: AD {}, DRMF {}, RMF {}, PCA {}'.format(np.mean(F_AD[:T+1]), np.mean(F_DRMF[:T+1]), np.mean(F_RMF[:T+1]), np.mean(F_PCA[:T+1])))
            print('max norm: AD {}, DRMF {}, RMF {}, PCA {}'.format(np.mean(max_AD[:T+1]), np.mean(max_DRMF[:T+1]), np.mean(max_RMF[:T+1]), np.mean(max_PCA[:T+1])))
            print('AUC: AD {}, DRMF {}, RMF {}, PCA {}'.format(np.mean(AUC_AD[:T+1]), np.mean(AUC_DRMF[:T+1]), np.mean(AUC_RMF[:T+1]), np.mean(AUC_PCA[:T+1])))


            if (T % 10 == 0 or T == num_exper - 1):
                print('In the step {}, saving data...'.format(T))
                if not os.path.exists(file_path):
                    os.makedirs(file_path)
                f = open(file_path+file_name, 'wb')
                np.save(f, parameters)
                np.save(f, AUC_opt)
                np.save(f, AUC_AD)
                np.save(f, AUC_DRMF)
                np.save(f, AUC_RMF)
                np.save(f, AUC_PCA)
                np.save(f, F_AD)
                np.save(f, F_DRMF)
                np.save(f, F_RMF)
                np.save(f, F_PCA)
                np.save(f, max_AD)
                np.save(f, max_DRMF)
                np.save(f, max_RMF)
                np.save(f, max_PCA)
                #self.plot_scatter()
            # if (T % 20 == 0 or T == num_exper - 1):
            #     print('In the step {}, saving data...'.format(T))
            #     if not os.path.exists(file_path):
            #         os.makedirs(file_path)
            #     f = open(file_path+file_name, 'wb')
            #     np.save(f, parameters)
            #     np.save(f, AUC_opt)
            #     np.save(f, AUC_AD)
            #     np.save(f, AUC_DRMF)
            #     self.plot_scatter()

    def plot_scatter(self, data = 'synthetic', file_path = "", file_name = ""):
        
        if (file_name == ""):
            file_name = self.file_name
            file_path = self.file_path

        f = open(file_path+file_name, 'rb')
        parameters = np.load(f)
        AUC_opt = np.load(f)
        AUC_AD = np.maximum(np.load(f), 0.5)
        AUC_PCA = np.maximum(np.load(f), 0.5)
        print(np.mean(AUC_opt))
        print(np.mean(AUC_AD))
        print(np.mean(AUC_PCA))
        print(len(parameters))

        plt.plot(AUC_AD, AUC_opt, 'bo', [0, 1], [0, 1], 'tab:orange', AUC_AD, AUC_PCA, 'go', linewidth=4.0)
        plt.xlim([0.5, 1])
        plt.ylim([0.5, 1])
        plt.xlabel(r'AUC of $\pi^{\mathrm{EW}}$')
        plt.ylabel(r'AUC')
        plt.legend([r'($\pi^{\mathrm{EW}}$, $\pi^{\mathrm{ideal}}$)', r'($\pi^{\mathrm{EW}}$, $\pi^{\mathrm{EW}}$)', r'($\pi^{\mathrm{EW}}$, Robust PCA)'])
        plt.savefig(file_path+'AUC-'+data+'.eps')
        plt.show()

This code can provide the synthetic setup with 1000 repetitions. 
Report four different algortihms on three metrics: ACU, Frobenious norm, and max norm. 

For Stable-PCP, we optimize lambda and mu for plotting a curve. We do so by first enumerate the ratio = lambda / mu, and then scaling for fitting the rank constraint. 

For RMC: the algorithm is implemented based on Stable-PCP, with projection on the max norm constraints ($|M|_{oo} \leq a, |S|_{oo} \leq a$)

For DRMF: we implement the algorithm with the maximal rank-constraint, and maximal p_a constraint. 

All metrices are measured based on the average. 

## Synthetic Experiment

In [ ]:
n = 100
r = 3
np.random.seed(3)
experiment = Experiment(n1=n, n2=n, r=r, mean_value=1, prob_observing = 0.5)
experiment.generate_data_for_scatter_plot(file_path = 'results/06-04-beer/', file_name = 'synthetic_data.npy', force=True)
#experiment.plot_scatter()
#experiment.real_data()
#experiment.generate_data_for_heat_map(l1=2, l2=2, force=False)

#experiment.plot_heat_map(file_path = 'results/05-20/', file_name = 'synthetic-n=100-r=3-heat_map_data')

In [ ]:
#r is 2, mean is 7.690840760524146, p_o is 0.8243170706184999, p_a is 0.224935479196669, alpha is 0.7110836066501481
#r is 1, mean is 5.8226985649490866, p_o is 0.7046509316108835, p_a is 0.04416386204681395, alpha is 0.09484103200079186


n = 100
r = 5
mean_value = 3
prob_observing = 0.7
np.random.seed(1)
experiment = Experiment(n1=n, n2=n, r=r, mean_value=mean_value, prob_observing = prob_observing)
noise = Exponential_noise_model(p = 0.1, exp_rate = 1/0.2, p_range = (0.01, 0.5), one_over_exp_range=(0.01, 0.8))
experiment.add_noise(noise)
experiment.ideal_ROC()
experiment.AD_algorithm(gamma = (n/r*0.5), r_constraint = r, debug=True, do_SVD = False, using_ideal_parameter=False)
experiment.PCA_with_M(experiment.M0)
#experiment.Robust_PCA(gamma = (n/r*0.5/2)*3, r_constraint=r, debug = True, num_points=100, l_range=(0.5, mean_value))
#print(noise_to_signal(experiment.M, experiment.M0*(1-noise.p+noise.p/noise.exp_rate), experiment.Ω))
experiment.plot_ROC()

## Real Experiment

In [ ]:
np.random.seed(1)
#if (experiment.data_type != 'beer_data'):
experiment = Experiment()
experiment.real_data()
print(experiment.mean_value)
noise = Exponential_noise_model(p = 0.04, exp_rate = 1/0.2, p_range = (0.01, 0.5), one_over_exp_range=(0.01, 0.8))
#experiment.add_noise(noise)
experiment.ideal_ROC()
experiment.AD_algorithm(gamma = 200, r_constraint = experiment.r, debug=True, do_SVD = False, using_ideal_parameter=False)
experiment.PCA_with_M(experiment.M0)
#experiment.Robust_PCA(gamma = 50, r_constraint=experiment.r)
#print(noise_to_signal(experiment.M, experiment.M0*(1-noise.p+noise.p/noise.exp_rate), experiment.Ω))
experiment.plot_ROC()